In [1]:
import os

### openBLAS will not work properly in jupyter notebook
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np
from scipy import stats as ss
import snapatac2 as snap
import scanpy as sc
import anndata as an
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors

/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/y2xie/miniconda3/envs/seurat/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Impor

## Subset siletti datasets by including cells in the matched dissection regions

In [6]:
rmeta = pd.read_csv("../../analysis/04.clustering/02.RNA/03.l3_clustering/MiniAtlas_RNA_merged_dual_filt_clean_metadata_250520.txt", 
                  sep = "\t")
np.unique(rmeta.region)

array(['A19', 'A25', 'A38', 'A46', 'A5-A7', 'CBL', 'CBV', 'ITG', 'Idg',
       'Ig', 'M1C', 'MEC', 'MTG', 'PN', 'Pro', 'TH-TL', 'V1C', 'V2'],
      dtype=object)

In [2]:
t1 = sc.read_h5ad("CELLxGene_rds/0bb62fec-0cf1-46e1-9d10-de65a6d4f814.h5ad")
t2 = sc.read_h5ad("CELLxGene_rds/9082ad42-b5ba-449f-ad10-1b988ac79eaa.h5ad")

In [25]:
# Step 1: subset t1.obs by region
roi_list = ["Human " + r for r in rmeta["region"].unique()]
qmeta1 = t1.obs[t1.obs["roi"].isin(roi_list)].copy()

# Step 2: keep supercluster_term with at least 20 cells
qterm1 = qmeta1["supercluster_term"].value_counts()
qterm1 = qterm1[qterm1 >= 20].index.tolist()

# Step 3: sample 100 cells per cluster_id, only for those in qterm1
qmeta1 = qmeta1[qmeta1["supercluster_term"].isin(qterm1)]

qcell1 = qmeta1.groupby("cluster_id", group_keys=False).apply(lambda x: x.sample(n=min(200, len(x)), random_state=1)).index.tolist()

# Repeat for t2
qmeta2 = t2.obs[t2.obs["roi"].isin(roi_list)].copy()

qterm2 = qmeta2["supercluster_term"].value_counts()
qterm2 = qterm2[qterm2 >= 20].index.tolist()
qmeta2 = qmeta2[qmeta2["supercluster_term"].isin(qterm2)]
qcell2 = qmeta2.groupby("cluster_id", group_keys=False).apply(lambda x: x.sample(n=min(200, len(x)), random_state=1)).index.tolist()

print(len(qcell1))
print(len(qcell2))

/tmp/ipykernel_749477/3676872699.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  qcell1 = qmeta1.groupby("cluster_id", group_keys=False).apply(lambda x: x.sample(n=min(200, len(x)), random_state=1)).index.tolist()


30926
7651


/tmp/ipykernel_749477/3676872699.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qcell1 = qmeta1.groupby("cluster_id", group_keys=False).apply(lambda x: x.sample(n=min(200, len(x)), random_state=1)).index.tolist()
/tmp/ipykernel_749477/3676872699.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  qcell2 = qmeta2.groupby("cluster_id", group_keys=False).apply(lambda x: x.sample(n=min(200, len(x)), random_state=1)).index.tolist()
/tmp/ipykernel_749477/3676872699.py:20: DeprecationWarning: DataFrameGroupB

In [28]:
t1 = t1[qcell1,:]
t2 = t2[qcell2,:]

In [32]:
t1.write("CELLxGene_rds/siletti_Neu_minialtas_subset.h5ad")
t2.write("CELLxGene_rds/siletti_NonN_minialtas_subset.h5ad")